collect all results into a spreadsheet

In [2]:
import sys
sys.path.append("../")
import os
import numpy as np
import nibabel as nb
import pandas as pd
import matplotlib.pylab as plt
from data.nifti_dataset import resample_nifti
from data.base_dataset import _roll2center_crop
from scipy.ndimage.measurements import center_of_mass

import DeepStrain.functions_collection as ff
import CMR_HFpEF_Analysis.Image_utils as util

main_path = '/mnt/mount_zc_NAS/Deepstrain'
data_path = '/mnt/mount_zc_NAS/HFpEF/data/HFpEF_data'
raw_spreadsheet = pd.read_excel(os.path.join(data_path, 'Patient_list', 'Important_HFpEF_Patient_list_unique_patient_w_readmission_finalized.xlsx' ))

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# define patient list:
index_range = range(0,50)
patient_list = raw_spreadsheet.iloc[index_range]
save_folder = os.path.join(main_path, 'results/result_collections')

# Spreadsheet for geometry measurements

In [50]:
result = []
for i in range(0, patient_list.shape[0]):
    patient_id_num = patient_list['OurID'].iloc[i]
    patient_id = ff.XX_to_ID_00XX(patient_id_num)
   
    # load geometry then:
    geometry_folder = os.path.join(main_path, 'results/geometry', patient_id)
    
    # circular index
    circular_index = np.load(os.path.join(geometry_folder, 'circular_index_collect.npy')).reshape(-1)  # 0:10
    
    # centers
    centers = np.load(os.path.join(geometry_folder, 'centers_collect.npy'), allow_pickle=True)
    center_delta_to_base = centers[1]
    center_delta_X_to_base = centers[1][:,0].reshape(-1) # 10:20
    center_delta_Y_to_base = centers[1][:,1].reshape(-1) # 20:30
    center_delta_Euclidean_to_base = np.sqrt(np.square(center_delta_X_to_base) + np.square(center_delta_Y_to_base)) # 30:40
    
    enclosed_area = np.asarray(centers[-1]).reshape(-1) # 40:41


    # axis len
    axis = np.load(os.path.join(geometry_folder, 'axis_len_collect.npy'), allow_pickle=True)
    major_axis = axis[:-1,0].reshape(-1) #41:51
    minor_axis = axis[:-1,1].reshape(-1) #51:61

    # combine these results together into a array of (num_parameters,)
    a = np.round(np.concatenate((circular_index, center_delta_X_to_base, center_delta_Y_to_base, center_delta_Euclidean_to_base, enclosed_area, major_axis, minor_axis), axis=0),3)
    a = [patient_id_num, patient_id] + a.tolist()
    
    result.append(a)

# save the result
name = ['circular_index', 'center_delta_X_to_base', 'center_delta_Y_to_base', 'center_delta_Euclidean_to_base', 'enclosed_area', 'major_axis', 'minor_axis']
column_list = ['OurID', 'PatientID']
for n in name:
    if n[0:3] != 'enc': # 'enclosed_area' is a single value
        for i in range(0,10):
            column_list.append(n + '_slice_' + str(i + 1))
    else:
        column_list.append(n)

df = pd.DataFrame(result, columns=column_list)
df_info = patient_list[['OurID','Final label if count surgery at day 0', 'Final label if not count surgery at day 0', 'ZC label', 'ED', 'ES']]

# merge df and df_info according to "OurID", df_info first then df
df = pd.merge(df_info, df, on='OurID')
# move patientID to the second column
column_to_move = 'PatientID'
target_position = 1 
columns = df.columns.tolist()
columns.remove(column_to_move)
columns.insert(target_position, column_to_move)
df = df[columns]

df.to_excel(os.path.join(save_folder, 'geometry_collection.xlsx'), index=False)


# Spreadsheet for strain measurements, ED and ES

In [51]:
result = []
for i in range(0, patient_list.shape[0]):
    patient_id_num = patient_list['OurID'].iloc[i]
    patient_id = ff.XX_to_ID_00XX(patient_id_num)
   
    # load strain first:
    strain_folder = os.path.join(main_path, 'results/strain', patient_id)
    # global strain
    global_strain = np.load(os.path.join(strain_folder, 'global_strain.npy'))
    global_radial_strain = np.asarray(global_strain[0]).reshape(-1)   # 0:1
    global_circumferential_strain = np.asarray(global_strain[1]).reshape(-1) # 1:2

    # layer strain
    layer_strain = np.load(os.path.join(strain_folder, 'layer_strain.npy'))
    layer_radial_strain = layer_strain[0,:].reshape(-1) # 2:5
    layer_circumferential_strain = layer_strain[1,:].reshape(-1) # 5:8

    # polar strain
    polar_strain = np.load(os.path.join(strain_folder, 'polar_strain.npy'), allow_pickle=True)
    Err_aha = np.asarray(polar_strain[5][:-1]).reshape(-1) # 8:24
    Ecc_aha = np.asarray(polar_strain[7][:-1]).reshape(-1) # 24:40

    # WTCI
    wtci = np.load(os.path.join(strain_folder, 'wtci.npy'), allow_pickle=True)
    wtci_aha = np.asarray(wtci[4][:-1]).reshape(-1) # 40:56

    # combine these results together into a array of (num_parameters,)
    a = np.round(np.concatenate((global_radial_strain, global_circumferential_strain, layer_radial_strain, layer_circumferential_strain, Ecc_aha, Err_aha, wtci_aha), axis=0),3)
    a = [patient_id_num, patient_id] + a.tolist()
    result.append(a)


# save the result
walls = ['basal_anterior', 'basal_anteroseptal', 'basal_inferoseptal', 'basal_inferior', 'basal_inferolateral', 'basal_anterolateral', 'mid_anterior', 'mid_anteroseptal', 'mid_inferoseptal', 'mid_inferior', 'mid_inferolateral', 'mid_anterolateral', 'apical_anterior', 'apical_septal', 'apical_inferior', 'apical_lateral']
column_list = ['OurID', 'PatientID', 'Err_global', 'Ecc_global', 'Err_base', 'Err_mid', 'Err_apex', 'Ecc_base', 'Ecc_mid', 'Ecc_apex' ]
for n in ['Err', 'Ecc','WTCI']:
    for w in walls:
        column_list.append(n + '_' + w)
print(column_list, len(column_list))

df = pd.DataFrame(result, columns=column_list)
df_info = patient_list[['OurID','Final label if count surgery at day 0', 'Final label if not count surgery at day 0', 'ZC label', 'ED', 'ES', 'LVEF']]

# merge df and df_info according to "OurID", df_info first then df
df = pd.merge(df_info, df, on='OurID')
# move patientID to the second column
column_to_move = 'PatientID'
target_position = 1 
columns = df.columns.tolist()
columns.remove(column_to_move)
columns.insert(target_position, column_to_move)
df = df[columns]

df.to_excel(os.path.join(save_folder, 'strain_ED_ES_collection.xlsx'), index=False)

['OurID', 'PatientID', 'Err_global', 'Ecc_global', 'Err_base', 'Err_mid', 'Err_apex', 'Ecc_base', 'Ecc_mid', 'Ecc_apex', 'Err_basal_anterior', 'Err_basal_anteroseptal', 'Err_basal_inferoseptal', 'Err_basal_inferior', 'Err_basal_inferolateral', 'Err_basal_anterolateral', 'Err_mid_anterior', 'Err_mid_anteroseptal', 'Err_mid_inferoseptal', 'Err_mid_inferior', 'Err_mid_inferolateral', 'Err_mid_anterolateral', 'Err_apical_anterior', 'Err_apical_septal', 'Err_apical_inferior', 'Err_apical_lateral', 'Ecc_basal_anterior', 'Ecc_basal_anteroseptal', 'Ecc_basal_inferoseptal', 'Ecc_basal_inferior', 'Ecc_basal_inferolateral', 'Ecc_basal_anterolateral', 'Ecc_mid_anterior', 'Ecc_mid_anteroseptal', 'Ecc_mid_inferoseptal', 'Ecc_mid_inferior', 'Ecc_mid_inferolateral', 'Ecc_mid_anterolateral', 'Ecc_apical_anterior', 'Ecc_apical_septal', 'Ecc_apical_inferior', 'Ecc_apical_lateral', 'WTCI_basal_anterior', 'WTCI_basal_anteroseptal', 'WTCI_basal_inferoseptal', 'WTCI_basal_inferior', 'WTCI_basal_inferolateral